<a href="https://colab.research.google.com/github/veeralakrishna/Utilities/blob/master/Plotly/07_PM_Precedence_Matrix_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>ProcessMining - Precedence Matrix</center></h1>

A precendence diagrams is a two-dimensional matrix showing the flows between activities. It can be contain different type values, by adjusting the type argument.

In [ ]:
pip install pm4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.3 MB/s 
  Created wheel for pyvis: filename=pyvis-0.2.1-py3-none-any.whl size=23688 sha256=50f1734ca0b0f483bf7b620d83494a2f45bbfddbd1666be753859936703f911e
  Stored in directory: /root/.cache/pip/wheels/2a/8f/04/6340d46afc74f59cc857a594ca1a2a14a1f4cbd4fd6c2e9306
  Created wheel for stringdist: filename=StringDist-1.0.9-cp37-cp37m-linux_x86_64.whl size=23605 sha256=f82827a07664ac4947d95b55dee7325ac3814a03292592590443b063688b0995
  Stored in directory: /root/.cache/pip/wheels/d7/9c/d4/63bc3d8931de0980b9e4a724dea290bb40b8b1b2bd6227c8da
Successfully built pyvis stringdist


In [1]:
import plotly.express as px
from textwrap import wrap

named_colorscales = px.colors.named_colorscales()

# named_colorscales

In [ ]:
import pandas as pd
import numpy as np
import itertools

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch


import pm4py
from pm4py.util import constants
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.statistics.traces.generic.log import case_statistics
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.visualization.graphs import visualizer as graphs_visualizer
from pm4py.algo.discovery.correlation_mining import algorithm as correlation_miner


pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
df = pm4py.format_dataframe(pd.read_csv('https://raw.githubusercontent.com/pm4py/pm4py-core/release/notebooks/data/running_example.csv', sep=';'), case_id='case_id',activity_key='activity', timestamp_key='timestamp')

In [ ]:
df = pd.read_csv("processmining.csv")


df['CREATE_DATE'] = pd.to_datetime(df['CREATE_DATE'], format="%d-%b-%y")

df['COMPLETION_DATE'] = pd.to_datetime(df['COMPLETION_DATE'], format="%d-%b-%y")


df = df.sort_values(by=['CREATE_DATE'])

# df.reset_index(inplace=True, drop=True)

data = pm4py.format_dataframe(df, "CLAIM_ID", "TASK", "CREATE_DATE")

log = log_converter.apply(data)

In [ ]:
# log = log_converter.apply(df)

In [ ]:



def get_antecedent_consequent(log):

    """
    Function to get antecedent and consequent from log

    Params:
        log         : pm4py log

    Returns:
    preced_df       : a pandas dataframe with antecedent $ consequent frequencies

    """

    d = pm4py.discover_dfg(log)[0]

    preced_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
    preced_df.rename(columns={"index" : "Antecedent,Consequent",
                              0 : "Absolute_Frequency"},
                     inplace=True)

    preced_df['Antecedent'], preced_df['Consequent'] = zip(*preced_df["Antecedent,Consequent"])

    # calculate relative frequencies from values in column 'Absolute_Frequency'
    preced_df['Relative_Frequency'] = (preced_df['Absolute_Frequency']/preced_df['Absolute_Frequency'].sum())*100

    preced_df["Relative_Frequency"] = preced_df["Relative_Frequency"].round(3)

    return preced_df

In [ ]:
preced_df = get_antecedent_consequent(log)

In [ ]:
["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

['plotly',
 'plotly_white',
 'plotly_dark',
 'ggplot2',
 'seaborn',
 'simple_white',
 'none']

In [ ]:
preced_df

# [["Antecedent", "Consequent", "Absolute_Frequency", "Relative_Frequency"]]

,"Antecedent,Consequent",Absolute_Frequency,Antecedent,Consequent,Relative_Frequency
0,"(Claim Handler - Review File, Claim Handler - ...",16,Claim Handler - Review File,Claim Handler - Review File,1.488
1,"(Claim Handler - Review File, Review New File)",8,Claim Handler - Review File,Review New File,0.744
2,"(Review New File, Complete Self Service FNOL -...",13,Review New File,Complete Self Service FNOL - Auto,1.209
3,"(Complete Self Service FNOL - Auto, Schedule I...",13,Complete Self Service FNOL - Auto,Schedule Inspection Assignment,1.209
4,"(Schedule Inspection Assignment, Order Police ...",4,Schedule Inspection Assignment,Order Police Report - Auto Express,0.372
...,...,...,...,...,...
502,"(Send at fault letter to First Party, Complete...",1,Send at fault letter to First Party,Complete Self Service FNOL - Auto,0.093
503,(Review Claim for Closure or Send Status Lette...,1,Review Claim for Closure or Send Status Letter,Check Issued - Review for Closure,0.093
504,"(Check Issued - Review for Closure, Express Fi...",1,Check Issued - Review for Closure,Express Financials Pending,0.093
505,"(Review New File, Send Diminished Value letter)",1,Review New File,Send Diminished Value letter,0.093


In [ ]:
preced_df["Relative_Frequency"].sum()

99.975

In [ ]:

import plotly.express as px


def plotly_precedence_matrix(df,
                             y="Antecedent",
                             x="Consequent",
                             freq="Absolute_Frequency",
                             template="plotly_white",
                             title_text="Precedence Matrix",
                             height=600,
                             width=1200,
                             marker_size=50
                             ):
    """
    Function to plot Precedence Matrix in Plotly

    Params:
        df(a dataframe)         : a pandas dataframe
        y(str)                  : Antecedent
        x(str)                  : Consequent
        freq(str)               : frequency -
                                  Absolute_Frequency or Relative_Frequency
        templete(str)           : Plotly templete
        height(int)             : Plotly figure height
        width(int)              : Plotly figure width
        marker_size(int)        : Marker size

    """

    fig = px.scatter(df,
                     y=y,
                     x=x,
                     color=freq,
                     text=freq,
                     custom_data=[freq],
                     color_continuous_scale="tealgrn"
                    # tealgrn, tealrose,teal,
                    )

    fig.update_traces(
        hovertemplate="<br>".join([
            "Antecedent: %{y}",
            "Consequent: %{x}",
            "Count: %{customdata[0]}"
        ]),
        name=""
    )

    fig.update_layout(
        template=template,
        # Figure title styling
        title= dict(
            text=title_text,
            x=0.5,
            y=0.95,
            font=dict(
                family="Rockwell",
                size=20,
                color='#000000'
                )
            ),
        xaxis_title=x,
        yaxis_title=y,
        font=dict(
            # family="Rockwell",
            size=14),
        # Fig height
        height=height,
        width=width,
        # Turn off legend
        showlegend=False,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
            )
        )

    fig.update_traces(marker=dict(size=marker_size,
                                  symbol='square',
                                  line=dict(width=2,
                                            color='DarkSlateGrey',
                                            )
                                  ),

                      textfont_color='black',
                      opacity=0.75,
                      textposition='middle center')

    fig.update_coloraxes(colorbar_title_font_size=14,
                         colorbar_title_side="right")
    fig.show()

In [ ]:
plotly_precedence_matrix(preced_df,
                         y="Antecedent",
                         x="Consequent",
                         freq="Relative_Frequency",
                         template="plotly_white",
                         title_text="Precedence Matrix",
                         marker_size=55
                        #  height=1200,
                        #  width=1500
                        )